# T1176 - Browser Extensions
Adversaries may abuse Internet browser extensions to establish persistent access to victim systems. Browser extensions or plugins are small programs that can add functionality and customize aspects of Internet browsers. They can be installed directly or through a browser's app store and generally have access and permissions to everything that the browser can access.(Citation: Wikipedia Browser Extension)(Citation: Chrome Extensions Definition)

Malicious extensions can be installed into a browser through malicious app store downloads masquerading as legitimate extensions, through social engineering, or by an adversary that has already compromised a system. Security can be limited on browser app stores so it may not be difficult for malicious extensions to defeat automated scanners.(Citation: Malicious Chrome Extension Numbers) Depending on the browser, adversaries may also manipulate an extension's update url to install updates from an adversary controlled server or manipulate the mobile configuration file to silently install additional extensions.

Previous to macOS 11, adversaries could silently install browser extensions via the command line using the <code>profiles</code> tool to install malicious <code>.mobileconfig</code> files. In macOS 11+, the use of the <code>profiles</code> tool can no longer install configuration profiles, however <code>.mobileconfig</code> files can be planted and installed with user interaction.(Citation: xorrior chrome extensions macOS)

Once the extension is installed, it can browse to websites in the background, steal all information that a user enters into a browser (including credentials), and be used as an installer for a RAT for persistence.(Citation: Chrome Extension Crypto Miner)(Citation: ICEBRG Chrome Extensions)(Citation: Banker Google Chrome Extension Steals Creds)(Citation: Catch All Chrome Extension)

There have also been instances of botnets using a persistent backdoor through malicious Chrome extensions.(Citation: Stantinko Botnet) There have also been similar examples of extensions being used for command & control.(Citation: Chrome Extension C2 Malware)

## Atomic Tests

### Atomic Test #1 - Chrome/Chromium (Developer Mode)Turn on Chrome/Chromium developer mode and Load Extension found in the src directory**Supported Platforms:** linux, windows, macosRun it with these steps!1. Navigate to [chrome://extensions](chrome://extensions) and
tick 'Developer Mode'.

2. Click 'Load unpacked extension...' and navigate to
[Browser_Extension](../t1176/src/)

3. Click 'Select'


### Atomic Test #2 - Chrome/Chromium (Chrome Web Store)Install the "Minimum Viable Malicious Extension" Chrome extension**Supported Platforms:** linux, windows, macosRun it with these steps!1. Navigate to https://chrome.google.com/webstore/detail/minimum-viable-malicious/odlpfdolehmhciiebahbpnaopneicend
in Chrome

2. Click 'Add to Chrome'


### Atomic Test #3 - FirefoxCreate a file called test.wma, with the duration of 30 seconds
**Supported Platforms:** linux, windows, macosRun it with these steps!1. Navigate to [about:debugging](about:debugging) and
click "Load Temporary Add-on"

2. Navigate to [manifest.json](./src/manifest.json)

3. Then click 'Open'


### Atomic Test #4 - Edge Chromium Addon - VPNAdversaries may use VPN extensions in an attempt to hide traffic sent from a compromised host. This will install one (of many) available VPNS in the Edge add-on store.
**Supported Platforms:** windows, macosRun it with these steps!1. Navigate to https://microsoftedge.microsoft.com/addons/detail/fjnehcbecaggobjholekjijaaekbnlgj
in Edge Chromium

2. Click 'Get'


### Atomic Test #5 - Google Chrome Load Unpacked Extension With Command LineThis test loads an unpacked extension in Google Chrome with the `--load-extension` parameter. This technique was previously used by the Grandoreiro malware to load a malicious extension that would capture the browsing history, steal cookies and other user information. Other malwares also leverage this technique to hijack searches, steal passwords, inject ads, and more.

References:
https://attack.mitre.org/techniques/T1176/
https://securityintelligence.com/posts/grandoreiro-malware-now-targeting-banks-in-spain/**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell

# Chromium
$chromium =  "https://commondatastorage.googleapis.com/chromium-browser-snapshots/Win_x64/1153778/chrome-win.zip"

# uBlock Origin Lite to test side-loading
$extension = "https://github.com/gorhill/uBlock/releases/download/uBOLite_0.1.23.6055/uBOLite_0.1.23.6055.chromium.mv3.zip"

Set-Location "$env:TEMP"

Set-Variable ProgressPreference SilentlyContinue
Invoke-WebRequest -URI $chromium -OutFile "$env:TEMP\chrome.zip"
Invoke-WebRequest -URI $extension -OutFile "$env:TEMP\extension.zip"


Expand-Archive chrome.zip -DestinationPath "$env:TEMP" -Force
Expand-Archive extension.zip -Force

Start-Process .\chrome-win\chrome.exe --load-extension="$env:TEMP\extension\" -PassThru```

In [ ]:
Invoke-AtomicTest T1176 -TestNumbers 5

#### Cleanup: 
```powershell
Set-Location "$env:TEMP"
Stop-Process -Name chrome -Force
Remove-Item .\chrome.zip, .\chrome-win, .\extension, .\extension.zip -Recurse -Force
Set-Variable ProgressPreference Continue```

In [ ]:
Invoke-AtomicTest T1176 -TestNumbers 5 -Cleanup

## Detection
Inventory and monitor browser extension installations that deviate from normal, expected, and benign extensions. Process and network monitoring can be used to detect browsers communicating with a C2 server. However, this may prove to be a difficult way of initially detecting a malicious extension depending on the nature and volume of the traffic it generates.

Monitor for any new items written to the Registry or PE files written to disk. That may correlate with browser extension installation.

On macOS, monitor the command line for usage of the profiles tool, such as <code>profiles install -type=configuration</code>. Additionally, all installed extensions maintain a <code>plist</code> file in the <code>/Library/Managed Preferences/username/</code> directory. Ensure all listed files are in alignment with approved extensions.(Citation: xorrior chrome extensions macOS)


## Shield Active Defense
### Baseline 
 Identify key system elements to establish a baseline and be prepared to reset a system to that baseline when necessary. 

 Identify elements of software and configuration critical to a set of objectives, define their proper values, and be prepared to reset a running system to its intended state.  
#### Opportunity
There is an opportunity to use tools and controls to stop an adversary's activity.
#### Use Case
A defender can force the removal of browser extensions that are not allowed by a corporate policy.
#### Procedures
Maintain a verified baseline firewall configuration and use that copy as a fallback if an adversary alters that information.
Maintain a verified list of group policies enforced on a system and use that copy if an adversary attempts to deviate from the baseline.
